In [1]:
"imports"

import csv 
import numpy as np 
import pandas as pd 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold,train_test_split, LeaveOneOut, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
import matplotlib.pyplot as plt 


"load data set"

dataset = pd.read_csv("C:/Users/nibha/Desktop/syria.csv") 
X = dataset.drop('status', axis=1) # seperate features 
y = dataset['status'] #seperate label

"convert categorical classes"

z=dataset['deathdate'] 
need_dummies=X.drop('deathdate', axis=1) 
data_with_dummies= pd.get_dummies(need_dummies) 
X_features = pd.concat([ data_with_dummies,z], axis=1) #combine categorical and non categorial features 


X_features=np.asarray(X_features)

y = y.astype('category') 

y_target = y.cat.codes 

y_target=np.asarray(y_target) #class0-civilian


FileNotFoundError: File b'C:/Users/nibha/Desktop/syria.csv' does not exist

In [ ]:
'method for contingeny and other values'

def con(t_value,samples,target_pred_prob,y_true):
    target_pred=[]
    for j in range(samples):
      
        threshold=target_pred_prob[j]
        if (threshold<= t_value):
            target_pred.insert(j,1)
        else:
            target_pred.insert(j,0)
        
    tn, fp, fn, tp = confusion_matrix(y_true,target_pred).ravel() 
    
    sens= tp/(tp+fn) #True positive rate
    spec=tn/(fp+tn)
    fpr=100-(spec) #100-specificity
    
    acc=(tp+tn)/(tp+tn+fp+fn) 
    F1= (2*sens*spec)/(sens+spec)   
    
    return([tn,fp,fn,tp,sens,spec,fpr,acc,F1,target_pred])




In [ ]:

"k fold"

"get epoch like curve"

kf = KFold(n_splits=5) 

def k_model_epoch(depth):
    target_pred=[]
    tree = DecisionTreeClassifier(max_depth = depth,random_state=1) 
    
    for train_index, test_index in kf.split(X_features):
    
        tree.fit(X_features[train_index], y_target[train_index])
        kftarget_pred_real = tree.predict(X_features[test_index])
        scores=cross_val_score(tree, X_features, y_target)
    avg_score=scores.mean()
    
    return (avg_score)


for i in range(1,12):
   
    k_model3= k_model_epoch(3)
    k_model4= k_model_epoch(4)
    k_model5= k_model_epoch(5)
    k_model6= k_model_epoch(6)
    k_model7= k_model_epoch(7)
    k_model8= k_model_epoch(8)    
    k_model9= k_model_epoch(9)
    k_model10= k_model_epoch(10)
    k_model11= k_model_epoch(11)
    k_model12= k_model_epoch(12)
    k_model13= k_model_epoch(13)
    k_model14= k_model_epoch(14)
    
x_depth= [3,4,5,6,7,8,9,10,11,12,13,14]

k_avg_score=([k_model3,k_model4,k_model5,k_model6,k_model7,k_model8,k_model9,k_model10
            ,k_model11,k_model12,k_model13,k_model14])


fig=plt.figure()
ax = fig.add_subplot(111) 
ax.plot(x_depth,k_avg_score,marker="o",c='green', linewidth=2, label='accuracy')
ax.set(title='Decision Tree Accuracy for K-Fold', ylabel='Avg Cross Val Score',xlabel='Depth')
ax.legend(loc='best') 
plt.grid()
plt.show()

"create tree after getting best depth"

tree1 = DecisionTreeClassifier(max_depth =5,random_state=1) 


for train_index, test_index in kf.split(X_features):
    
    tree1.fit(X_features[train_index], y_target[train_index])
    kftarget_pred_real = tree1.predict(X_features[test_index])
    kftarget_pred_prob = tree1.predict_proba(X_features)[:,0]

samples=len(X_features)
kf_thresh0=con(0,samples,kftarget_pred_prob,y_target)
kf_thresh1=con(0.25,samples,kftarget_pred_prob,y_target)
kf_thresh2=con(0.5,samples,kftarget_pred_prob,y_target)
kf_thresh3=con(0.75,samples,kftarget_pred_prob,y_target)
kf_thresh4=con(1,samples,kftarget_pred_prob,y_target)

"ROC Curve"

x_k_fpr=[kf_thresh0[6],kf_thresh1[6],kf_thresh2[6],kf_thresh3[6],kf_thresh4[6]] #fpr or 100-specificty
# print(x_k_fpr)
y_k_tpr=[kf_thresh0[4],kf_thresh1[4],kf_thresh2[4],kf_thresh3[4],kf_thresh4[4]] #sensitivity
# print(y_k_tpr)
k_auc=roc_auc_score(y_target,kftarget_pred_prob)
# print(k_auc)

plt.title('ROC Curve for Decision Tree K Fold')
plt.plot(x_k_fpr,y_k_tpr, color='aqua',lw=2, label='(AUC = %0.2f)' % k_auc)
plt.legend(loc = 'lower right')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

# print(kf_thresh0[0],kf_thresh0[1],kf_thresh0[2],kf_thresh0[3],kf_thresh0[4],
#       kf_thresh0[5],kf_thresh0[6],kf_thresh0[7])
# output >3767 0 1225 5 0.00406504065041 1.0 0.754852911747 0.0080971659919 #tn,fp,fn,tp,sens,spec,acc,F1,
# print(kf_thresh1[0],kf_thresh1[1],kf_thresh1[2],kf_thresh1[3],kf_thresh1[4]
#       ,kf_thresh1[5],kf_thresh1[6],kf_thresh1[7])
# output>3719 48 1039 191 0.155284552846 0.9872577648 0.782469481689 0.2683592
# print(kf_thresh2[0],kf_thresh2[1],kf_thresh2[2],kf_thresh2[3],kf_thresh2[4]
#        ,kf_thresh2[5],kf_thresh2[6],kf_thresh2[7])
# output>3336 431 433 797 0.647967479675 0.88558534643 0.827096257755 0.748367444792
# print(kf_thresh3[0],kf_thresh3[1],kf_thresh3[2],kf_thresh3[3],kf_thresh3[4]
#        ,kf_thresh3[5],kf_thresh3[6],kf_thresh3[7])
# output>3052 715 245 985 0.80081300813 0.81019378816 0.807884730839 0.805476086332
# print(kf_thresh4[0],kf_thresh4[1],kf_thresh4[2],kf_thresh4[3],kf_thresh4[4]
#        ,kf_thresh4[5],kf_thresh4[6],kf_thresh4[7])
# output>0 3767 0 1230 1.0 0.0 0.246147688613 0.0

In [ ]:
"Leave one out"

loo = LeaveOneOut()
loo.get_n_splits(X_features)

"epoch curve"

def l_model_epoch(depth):
    target_pred=[]
    tree = DecisionTreeClassifier(max_depth = depth,random_state=1) 
    
    for train_index, test_index in loo.split(X_features):
    
        tree.fit(X_features[train_index], y_target[train_index])
        lootarget_pred_real = tree.predict(X_features[test_index])
        scores=cross_val_score(tree, X_features, y_target)
        avg_score=scores.mean()   
      
    return (avg_score)


for i in range(1,12):
   
    l_model3= np.asarray([l_model_epoch(3)])
    l_model4= np.asarray([l_model_epoch(4)])
    l_model5= np.asarray([l_model_epoch(5)])
    l_model6= np.asarray([l_model_epoch(6)])
    l_model7= np.asarray([l_model_epoch(7)])
    l_model8= np.asarray([l_model_epoch(8)])    
    l_model9= np.asarray([l_model_epoch(9)])
    l_model10= np.asarray([l_model_epoch(10)])
    l_model11= np.asarray([l_model_epoch(11)])
    l_model12= np.asarray([l_model_epoch(12)])
    l_model13= np.asarray([l_model_epoch(13)])
    l_model14= np.asarray([l_model_epoch(14)])
    
# x_depth= [3,4,5,6,7,8,9,10,11,12]

l_avg_score=([l_model3,l_model4,l_model5,l_model6,k_model7,l_model8,l_model9,l_model10
            ,l_model11,l_model12,l_model13,l_model14])

fig=plt.figure()
ax = fig.add_subplot(111) 

ax.plot(x_depth,l_avg_score, marker="o",c='blue', linewidth=2,label='accuracy')
ax.set(title='Decision Tree Accuracy for LOO', ylabel='Accuracy',xlabel='Depth')
ax.legend(loc='best') 
plt.grid()
plt.show()

# "create tree using loo"

tree2 = DecisionTreeClassifier(max_depth =5,random_state=1) 

for train_index, test_index in loo.split(X_features):
    
    tree2.fit(X_features[train_index], y_target[train_index])
    lootarget_pred_real = tree2.predict(X_features[test_index])
    
    

lootarget_pred_prob = tree2.predict_proba(X_features)[:,0]

samples=len(X_features)
loo_thresh0=con(0,samples,lootarget_pred_prob,y_target)
loo_thresh1=con(0.25,samples,lootarget_pred_prob,y_target)
loo_thresh2=con(0.5,samples,lootarget_pred_prob,y_target)
loo_thresh3=con(0.75,samples,lootarget_pred_prob,y_target)
loo_thresh4=con(1,samples,lootarget_pred_prob,y_target)

"ROC CURVE"

x_l_fpr=[loo_thresh0[6],loo_thresh1[6],loo_thresh2[6],loo_thresh3[6],loo_thresh4[6]] #fpr or 100-specificty
print(x_l_fpr)
y_l_tpr=[loo_thresh0[4],loo_thresh1[4],loo_thresh2[4],loo_thresh3[4],loo_thresh4[4]] #sensitivity
print(y_l_tpr)
l_auc=roc_auc_score(y_target,lootarget_pred_prob)
print(l_auc)

plt.title('ROC Curve for Decision Tree LOO')
plt.plot(x_l_fpr,y_l_tpr, color='deeppink',lw=2, label='(AUC = %0.2f)' % l_auc)
plt.legend(loc = 'lower right')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

# print(loo_thresh0[0],loo_thresh0[1],loo_thresh0[2],loo_thresh0[3],loo_thresh0[4],
#       loo_thresh0[5],loo_thresh0[6],loo_thresh0[7])
# # output >3767 0 1227 3 0.00243902439024 1.0 0.754452671603 0.00486618004866#tn,fp,fn,tp,sens,spec,acc,F1,
# print(loo_thresh1[0],loo_thresh1[1],loo_thresh1[2],loo_thresh1[3],loo_thresh1[4]
#       ,loo_thresh1[5],loo_thresh1[6],loo_thresh1[7])
# # output>3718 49 1034 196 0.159349593496 0.986992301566 0.783269961977 0.274397756403
# print(loo_thresh2[0],loo_thresh2[1],loo_thresh2[2],loo_thresh2[3],loo_thresh2[4]
#        ,loo_thresh2[5],loo_thresh2[6],loo_thresh2[7])
# # output>3332 431 423 797 0.656097560976 0.884523493496 0.828296978187 0.753376315381
# print(loo_thresh3[0],loo_thresh3[1],loo_thresh3[2],loo_thresh3[3],loo_thresh3[4]
#        ,loo_thresh3[5],loo_thresh3[6],loo_thresh3[7])
# # output>3054 715 241 985 0.80406504065 0.810724714627 0.809085451271 0.807381144811
# print(loo_thresh4[0],loo_thresh4[1],loo_thresh4[2],loo_thresh4[3],loo_thresh4[4]
#        ,loo_thresh4[5],loo_thresh4[6],loo_thresh4[7])
# # output>0 3767 0 1230 1.0 0.0 0.246147688613 0.0


In [ ]:
    
"3 tier scheme "
"epochh like curve"

# split data into train+validation set and test set
X_trainval, X_test, y_trainval, y_test = train_test_split(X_features, y_target, test_size= 0.2,random_state=2)

#split train+validation set into train set and validation set 
X_train, X_valid, y_train, y_valid =train_test_split(X_trainval, y_trainval, test_size= 0.2,random_state=2)

    def t_model_epoch(depth):
    
        tree=DecisionTreeClassifier(max_depth = depth,random_state=1)
        tree.fit(X_train, y_train) # train on train+validation set
        tree.fit(X_trainval, y_trainval) #train on training set
        train_acc=tree.score(X_trainval, y_trainval)
        test_acc=tree.score(X_test,y_test)       
        return ([train_acc,test_acc])

    for i in range(1,12):

        t_model3= np.asarray([t_model_epoch(3)])
        t_model4= np.asarray([t_model_epoch(4)])
        t_model5= np.asarray([t_model_epoch(5)])
        t_model6= np.asarray([t_model_epoch(6)])
        t_model7= np.asarray([t_model_epoch(7)])
        t_model8= np.asarray([t_model_epoch(8)])    
        t_model9= np.asarray([t_model_epoch(9)])
        t_model10= np.asarray([t_model_epoch(10)])
        t_model11= np.asarray([t_model_epoch(11)])
        t_model12= np.asarray([t_model_epoch(12)])
        t_model13= np.asarray([t_model_epoch(13)])
        t_model14= np.asarray([t_model_epoch(14)])
    
x_depth= [3,4,5,6,7,8,9,10,11,12,13,14]

y_train_acc=([t_model3[0,0],t_model4[0,0],t_model5[0,0],t_model6[0,0],t_model7[0,0],t_model8[0,0]
            ,t_model9[0,0],t_model10[0,0],t_model11[0,0],t_model12[0,0],t_model13[0,0],t_model14[0,0],])
y_test_acc=([t_model3[0,1],t_model4[0,1],t_model5[0,1],t_model6[0,1],t_model7[0,1],t_model8[0,1]
            ,t_model9[0,1],t_model10[0,1],t_model11[0,1],t_model12[0,1],t_model13[0,1],t_model14[0,1]])

fig=plt.figure()
ax = fig.add_subplot(111) 
ax.plot(x_depth,y_train_acc,marker="o",c='green', linewidth=2, label='training accuracy')
ax.plot(x_depth,y_test_acc, marker="o",c='blue', linewidth=2,label='testing accuracy')
ax.set(title='Decision Tree Accuracy for Tier', ylabel='Accuracy',xlabel='Depth')
ax.legend(loc='best') 
plt.grid()
plt.show()

"create tree for tier"
tree3 = DecisionTreeClassifier(max_depth =5,random_state=1) 

tree3.fit(X_train, y_train) # train on train+validation set
tree3.fit(X_trainval, y_trainval) #train on training set


tiertarget_pred_real=tree3.predict(X_test)
tiertarget_pred_prob = tree3.predict_proba(X_test)[:,0]

tsamples=len(X_test)
tier_thresh0=con(0,tsamples,tiertarget_pred_prob,y_test)
tier_thresh1=con(0.25,tsamples,tiertarget_pred_prob,y_test)
tier_thresh2=con(0.5,tsamples,tiertarget_pred_prob,y_test)
tier_thresh3=con(0.75,tsamples,tiertarget_pred_prob,y_test)
tier_thresh4=con(1,tsamples,tiertarget_pred_prob,y_test)

"ROC CURVE"

x_t_fpr=[tier_thresh0[6],tier_thresh1[6],tier_thresh2[6],tier_thresh3[6],tier_thresh4[6]] #fpr or 100-specificty
print(x_t_fpr)
y_t_tpr=[tier_thresh0[4],tier_thresh1[4],tier_thresh2[4],tier_thresh3[4],tier_thresh4[4]] #sensitivity
print(y_t_tpr)
t_auc=roc_auc_score(y_test,tiertarget_pred_prob)
print(t_auc)

plt.title('ROC Curve for Decision Tree Tier Scheme')
plt.plot(x_t_fpr,y_t_tpr, color='cornflowerblue',lw=2, label='(AUC = %0.2f)' % t_auc)
plt.legend(loc = 'lower right')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

# print(tier_thresh0[0],tier_thresh0[1],tier_thresh0[2],tier_thresh0[3],tier_thresh0[4],
#       tier_thresh0[5],tier_thresh0[6],tier_thresh0[7])
# output >758 0 242 0 0.0 1.0 0.758 0.0#tn,fp,fn,tp,sens,spec,acc,F1,
# print(tier_thresh1[0],tier_thresh1[1],tier_thresh1[2],tier_thresh1[3],tier_thresh1[4],
#       tier_thresh1[5],tier_thresh1[6],tier_thresh1[7])
# output>758 0 242 0 0.0 1.0 0.758 0.0
# print(tier_thresh2[0],tier_thresh2[1],tier_thresh2[2],tier_thresh2[3],tier_thresh2[4]
#        ,tier_thresh2[5],tier_thresh2[6],tier_thresh2[7])
# output>676 82 89 153 0.632231404959 0.891820580475 0.829 0.739918301939
# print(tier_thresh3[0],tier_thresh3[1],tier_thresh3[2],tier_thresh3[3],tier_thresh3[4]
#        ,tier_thresh3[5],tier_thresh3[6],tier_thresh3[7])
# output>611 147 45 197 0.814049586777 0.806068601583 0.808 0.810039436316
# print(tier_thresh4[0],tier_thresh4[1],tier_thresh4[2],tier_thresh4[3],tier_thresh4[4]
#        ,tier_thresh4[5],tier_thresh4[6],tier_thresh4[7])
# output>0 758 0 242 1.0 0.0 0.242 0.0


In [ ]:
"bar graph for kfold"
groups1 ={'TP':[kf_thresh0[3],kf_thresh1[3],kf_thresh2[3],kf_thresh3[3],kf_thresh4[3]],
          'TN':[kf_thresh0[0],kf_thresh1[0],kf_thresh2[0],kf_thresh3[0],kf_thresh4[0]], 
          'FP':[kf_thresh0[1],kf_thresh1[1],kf_thresh2[1],kf_thresh3[1],kf_thresh4[1]], 
          'FN':[kf_thresh0[2],kf_thresh1[2],kf_thresh2[2],kf_thresh3[2],kf_thresh4[2]]}

df1=pd.DataFrame(groups1,columns= ['TP', 'TN', 'FP', 'FN']).T

ax1 = df1.plot(kind='bar', title='Threshold Effect on DT Contingency for K-Fold')
ax1.legend(['T=0','T=0.25','T=0.5','T=0.75','T=1']);

"bar graph for loo"

groups2 ={'TP':[loo_thresh0[3],loo_thresh1[3],loo_thresh2[3],loo_thresh3[3],loo_thresh4[3]],
          'TN':[loo_thresh0[0],loo_thresh1[0],loo_thresh2[0],loo_thresh3[0],loo_thresh4[0]], 
          'FP':[loo_thresh0[1],loo_thresh1[1],loo_thresh2[1],loo_thresh3[1],loo_thresh4[1]], 
          'FN':[loo_thresh0[2],loo_thresh1[2],loo_thresh2[2],loo_thresh3[2],loo_thresh4[2]]}

df2=pd.DataFrame(groups2,columns= ['TP', 'TN', 'FP', 'FN']).T

ax2 = df2.plot(kind='bar', title='Threshold Effect on DT Contingency for Leave One Out') 
ax2.legend(['T=0','T=0.25','T=0.5','T=0.75','T=1']);

"bar graph for tier"

groups3 ={'TP':[tier_thresh0[3],tier_thresh1[3],tier_thresh2[3],tier_thresh3[3],tier_thresh4[3]], 
          'TN':[tier_thresh0[0],tier_thresh1[0],tier_thresh2[0],tier_thresh3[0],tier_thresh4[0]], 
          'FP':[tier_thresh0[1],tier_thresh1[1],tier_thresh2[1],tier_thresh3[1],tier_thresh4[1]], 
          'FN':[tier_thresh0[2],tier_thresh1[2],tier_thresh2[2],tier_thresh3[2],tier_thresh4[2]]}

# print(tier_thresh0[3],tier_thresh1[3],tier_thresh2[3],tier_thresh3[3],tier_thresh4[3]) 
# print(tier_thresh0[0],tier_thresh1[0],tier_thresh2[0],tier_thresh3[0],tier_thresh4[0]) 
# print(tier_thresh0[1],tier_thresh1[1],tier_thresh2[1],tier_thresh3[1],tier_thresh4[1]) 
# print(tier_thresh0[2],tier_thresh1[2],tier_thresh2[2],tier_thresh3[2],tier_thresh4[2])

df3=pd.DataFrame(groups3,columns= ['TP', 'TN', 'FP', 'FN']).T

ax3 = df3.plot(kind='bar', title='Threshold Effect on DT Contingency for Tier') 
ax3.legend(['T=0','T=0.25','T=0.5','T=0.75','T=1']);



groups1 ={'TP':[kf_thresh0[3],kf_thresh1[3],kf_thresh2[3],kf_thresh3[3],kf_thresh4[3]],
          'TN':[kf_thresh0[0],kf_thresh1[0],kf_thresh2[0],kf_thresh3[0],kf_thresh4[0]],
          'FP':[kf_thresh0[1],kf_thresh1[1],kf_thresh2[1],kf_thresh3[1],kf_thresh4[1]],
           'FN':[kf_thresh0[2],kf_thresh1[2],kf_thresh2[2],kf_thresh3[2],kf_thresh4[2]]}
                                           

df1=pd.DataFrame(groups1,columns= ['TP', 'TN', 'FP', 'FN']).T

ax1 = df1.plot(kind='bar', title='Threshold Effect on DT Contingency for K-Fold')
ax1.legend(['T=0','T=0.25','T=0.5','T=0.75','T=1']);

"bar graph for loo"

groups2 ={'TP':[loo_thresh0[3],loo_thresh1[3],loo_thresh2[3],loo_thresh3[3],loo_thresh4[3]],
          'TN':[loo_thresh0[0],loo_thresh1[0],loo_thresh2[0],loo_thresh3[0],loo_thresh4[0]],
          'FP':[loo_thresh0[1],loo_thresh1[1],loo_thresh2[1],loo_thresh3[1],loo_thresh4[1]],
           'FN':[loo_thresh0[2],loo_thresh1[2],loo_thresh2[2],loo_thresh3[2],loo_thresh4[2]]}

df2=pd.DataFrame(groups2,columns= ['TP', 'TN', 'FP', 'FN']).T


ax2 = df2.plot(kind='bar', title='Threshold Effect on DT Contingency for Leave One Out')
ax2.legend(['T=0','T=0.25','T=0.5','T=0.75','T=1']);

"bar graph for tier"

groups3 ={'TP':[tier_thresh0[3],tier_thresh1[3],tier_thresh2[3],tier_thresh3[3],tier_thresh4[3]],
          'TN':[tier_thresh0[0],tier_thresh1[0],tier_thresh2[0],tier_thresh3[0],tier_thresh4[0]],
          'FP':[tier_thresh0[1],tier_thresh1[1],tier_thresh2[1],tier_thresh3[1],tier_thresh4[1]],
           'FN':[tier_thresh0[2],tier_thresh1[2],tier_thresh2[2],tier_thresh3[2],tier_thresh4[2]]}
                                           
print(tier_thresh0[3],tier_thresh1[3],tier_thresh2[3],tier_thresh3[3],tier_thresh4[3])
print(tier_thresh0[0],tier_thresh1[0],tier_thresh2[0],tier_thresh3[0],tier_thresh4[0])
print(tier_thresh0[1],tier_thresh1[1],tier_thresh2[1],tier_thresh3[1],tier_thresh4[1])
print(tier_thresh0[2],tier_thresh1[2],tier_thresh2[2],tier_thresh3[2],tier_thresh4[2])

df3=pd.DataFrame(groups3,columns= ['TP', 'TN', 'FP', 'FN']).T



ax3 = df3.plot(kind='bar', title='Threshold Effect on DT Contingency for Tier')
ax3.legend(['T=0','T=0.25','T=0.5','T=0.75','T=1']);


In [ ]:
"bar graph for features"
 pd.value_counts(dataset['age'].values, sort=False))
# Child      580
# Adult     4417
display(pd.value_counts(dataset['gender'].values, sort=False))
# Female     609
# Male      4388
display(pd.value_counts(dataset['deathcause'].values, sort=False))
# Hasakeh                  32
# Other Nationalities      18
# Raqqa                    51
# Idlib                   895
# Homs                    325
# Damascus               1852
# Quneitra                 27
# Hama                    159
# Daraa                   295
# Lattakia                 50
# Aleppo                  930
# Deir Ezzor              363

display(pd.value_counts(dataset['status'].values, sort=False))
Field Execution                     425
Shooting                           1807
Explosion                            78
Un-allowed to seek Medical help      17
Detention                           154
Shelling                           2458
Kidnapping                           58
# display(pd.value_counts(dataset['status'].values, sort=False))
Non-Civilian    1230
Civilian        3767
# display(pd.value_counts(dataset['deathdate'].values, sort=False))
116
41200    229
41204    138
41208    129
41212    283
41216    201
41220    181
41197    121
41201    140
41205    177
41209     94
41213    160
41217    209
41221    139
41225    236
41194     77
41198    143
41202    157
41206    152
41210    161
41214    167
41218    191
41222    152
41226    173
41195    164
41199    175
41203    144
41207    110
41215    172
41219    160
41223    112
41227     34

features ={'Age':[],
          'TN':[kf_thresh0[0],kf_thresh1[0],kf_thresh2[0],kf_thresh3[0],kf_thresh4[0]],
          'FP':[kf_thresh0[1],kf_thresh1[1],kf_thresh2[1],kf_thresh3[1],kf_thresh4[1]],
           'FN':[kf_thresh0[2],kf_thresh1[2],kf_thresh2[2],kf_thresh3[2],kf_thresh4[2]]}
                                           
# print(kf_thresh0[3],kf_thresh1[3],kf_thresh2[3],kf_thresh3[3],kf_thresh4[3])
# print(kf_thresh0[0],kf_thresh1[0],kf_thresh2[0],kf_thresh3[0],kf_thresh4[0])
# print(kf_thresh0[1],kf_thresh1[1],kf_thresh2[1],kf_thresh3[1],kf_thresh4[1])
# print(kf_thresh0[2],kf_thresh1[2],kf_thresh2[2],kf_thresh3[2],kf_thresh4[2])

# Convert data to pandas DataFrame.
df1=pd.DataFrame(groups1,columns= ['TP', 'TN', 'FP', 'FN']).T

ax1 = df1.plot(kind='bar', title='Threshold Effect on DT Contingency for K-Fold')
ax1.legend(['T=0','T=0.25','T=0.5','T=0.75','T=1']);